# Torrent To Google Drive Downloader v4.1

### Mount Google Drive
To stream files we need to mount Google Drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

###Dependency
https://www.libtorrent.org/

In [ ]:
!python -m pip install lbry-libtorrent

###Timeout Prevention

In [ ]:
#@markdown <br><center><img src='https://miro.medium.com/max/2036/1*GW4cqNv-SIQ9BGGhssL1bQ.png' height="100" alt="netdata"/></center>
#@title <center><b>Time Out Preventer (Advanced) </b></center></strong>
%%capture
AUTO_RECONNECT = True #@param {type:"boolean"}
#@markdown **Run this code to prevent Google Colab from Timeout**
from os import makedirs
makedirs("/root/.config/rclone", exist_ok = True)
if AUTO_RECONNECT:
  import IPython
  from google.colab import output

  display(IPython.display.Javascript('''
  function ClickConnect(){
    btn = document.querySelector("colab-connect-button")
    if (btn != null){
      console.log("Click colab-connect-button"); 
      btn.click() 
      }
    
    btn = document.getElementById('ok')
    if (btn != null){
      console.log("Click reconnect"); 
      btn.click() 
      }
    }
    
  setInterval(ClickConnect,60000)
  '''))

### Code to download torrent

In [ ]:
#@title Torrent Downloader { form-width: "400px" }
#@markdown Input the magnet link below:
import libtorrent as lt
import time
import datetime
 
def download_torrent():
    ses = lt.session()
    ses.listen_on(6881, 6891)
    link = "" #@param {type:"string"}
    print(link)
 
    handle = lt.add_magnet_uri(ses, link, params)
    # change the 0 to a 1 to download sequential - this sequential option is only if you selected zip. If not,
    # scroll farther down.
    handle.set_sequential_download(0)
 
    ses.start_dht() 
    begin = time.time()
    print(datetime.datetime.now())
 
    print('Downloading Metadata...')
    while not handle.has_metadata():
        time.sleep(1)
    print('Got Metadata, Starting Torrent Download...')
 
    print("Starting", handle.name())
 
    while handle.status().state != lt.torrent_status.seeding:
        s = handle.status()
        state_str = ['queued', 'checking', 'downloading metadata',
                     'downloading', 'finished', 'seeding', 'allocating']
        print('%.2f%% complete (down: %.1f kb/s up: %.1f kB/s peers: %d) %s ' % \
              (s.progress * 100, s.download_rate / 1000, s.upload_rate / 1000, \
               s.num_peers, state_str[s.state]))
        time.sleep(5)
 
    end = time.time()
    print(handle.name(), "COMPLETE")
 
    print("Elapsed Time: ", int((end - begin) // 60), "min :", int((end - begin) % 60), "sec")
 
    print(datetime.datetime.now())
 
 #@markdown Input "Y"/"y" if you want to zip the files you want to download. Leave it blank you don't.
zipp = "" #@param {type:"string"}

#@markdown If you want to zip, input zip name: 
if zipp == 'Y' or zipp == 'y':
    zip_name = "" #@param {type:"string"}
    import shutil
 
    params = {
        'save_path': '/content/temp/',
        'storage_mode': lt.storage_mode_t(2),
    }
    download_torrent()
 
    print("now time to zip")
 
    print("Zipping ...")
    final_zip_directory = '/content/drive/Shareddrives/Mediastic2/Download/' + zip_name
    shutil.make_archive(final_zip_directory, 'zip', '/content/temp/')
    shutil.rmtree('/content/temp/')
 
else:
    params = {
        'save_path': '/content/drive/Shareddrives/Mediastic2/Download/',
        'storage_mode': lt.storage_mode_t(2),
    }
    download_torrent()
 
print('\nALL DONE!')